In [1]:
# Dashbords Datasets
Sales_full = 'Sales_full'
Sales_mini = 'Sales_mini'
Sales_mini = 'Sales_mini'
FBA_Fees='FBA Fees'
Current_price = 'Current_price'
Products_list = 'List_Of_All_Products.xlsx'

In [2]:
from os import listdir
from os.path import isfile, join
from openpyxl import load_workbook
import os
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from prettytable import PrettyTable
from google.colab import drive

In [3]:
drive.mount('/content/drive')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content'

# Required Data

## Sales Data

In [4]:
root_path = r"/content/drive/My Drive/Sales Data/Sales Analysis"
os.chdir(root_path)
onlyfiles = [f for f in listdir(root_path) if isfile(join(root_path, f))]
df = pd.DataFrame()
for i in onlyfiles[:-1]:
    df_tt = pd.read_excel(i)
    df = pd.concat([df, df_tt])

numeric_columns = df.select_dtypes(include='float').columns
df[numeric_columns] = df[numeric_columns].round(2)
df.reset_index(inplace = True, drop = True)

> Now, we have a dataframe **df** which is contains all the sales analysis data

## Importing The Products List

In [5]:
root_path = r"/content/drive/My Drive/Sales Data"
os.chdir(root_path)
df_n = pd.read_excel(Products_list)
df_n.rename(columns = {'ASINS': 'ASIN'}, inplace = True)

> Now, we have a dataframe **df_n** which contains the products list

In [6]:
df_t = pd.DataFrame()
for i in range(0, df.shape[0]):
    try:
        df_t = df_n[df_n['ASIN'] == df.loc[i,'ASIN']]
        df_t.reset_index(inplace = True, drop = True)
        df.loc[i, 'PRODUCTS'] =df_t['Product Name'][0]
    except:
        pass

# My Functions

### Price Points Classification columns

In [7]:
def SalesProfit(df_S):
    """
    This function takes the sales analysis DataFrame and returns it with a classification column for each price point based on the average net profit.
    The classification column will have one of the following values:
    - More Sales
    - Less Sales
    - No Difference
    """

    df_t = pd.DataFrame()
    df_final = pd.DataFrame()

    # Loop for comparing on the ASIN level
    for a in df_S['ASIN'].unique():
        df_t = df_S[df_S['ASIN'] == a]
        df_t.reset_index(inplace = True)
        df_t.drop(columns = 'index', inplace = True)

        # Loop to classify based on the price point
        for i in range(0, df_t.shape[0]):
            df_t = df_t.copy()
            if i == 0:
                df_t.loc[i,'Result in Sales'] = None
                df_t.loc[i,'Result in Profit'] = None
            else:
                if df_t.loc[old_i,'Average Sales per day'] > df_t.loc[i,'Average Sales per day']:
                    df_t.loc[i,'Result in Sales'] = 'Less Sales'
                elif df_t.loc[old_i,'Average Sales per day'] < df_t.loc[i,'Average Sales per day']:
                    df_t.loc[i,'Result in Sales'] = 'More Sales'
                else:
                    df_t.loc[i,'Result in Sales'] = 'No difference'
                if df_t.loc[old_i,'Average Daily Profit'] > df_t.loc[i,'Average Daily Profit']:
                    df_t.loc[i,'Result in Profit'] = 'Less Profit'
                elif df_t.loc[old_i,'Average Daily Profit'] < df_t.loc[i,'Average Daily Profit']:
                    df_t.loc[i,'Result in Profit'] = 'More Profit'
                else:
                    df_t = df_t.copy()
                    df_t.loc[i,'Result in Profit'] = 'No difference'
            # old_i means the pervious transcations
            old_i = i
        # After all the ASIN's rows done, we will add to our final dataframe
        df_final = pd.concat([df_final, df_t])
        df_final.reset_index(inplace = True, drop = True)
    return df_final

### Getting The Week Of Each Month Function

In [8]:
def week_of_month(date):
    first_day = date.replace(day=1)  # Get the first day of the month
    return (date.day + first_day.weekday()) // 7 + 1

## The Sales Data column Generation Function

### Teansfers Rows To Table Function

In [9]:
def generate_pretty_table(row):
    """
    Getting a row and put its data in a small table
    Input: row - the row needed to be a table
    > Row should contains the following metrics:
        >> **Price point**
        >> **No. of days**
        >> **Units Sold**
        >> **Average units sold per day**
        >> **Average Daily Profit**
        >> **Average Daily Profit Without COGS**
    output: table in a string shape

    """
    table = PrettyTable()
    table.field_names = ["Metric", "Value"]
    table.add_row(["Price point", row["Price"]])
    table.add_row(["Days observation", row["No. of days"]])
    table.add_row(["Units Sold", row["Units Sold"]])
    table.add_row(["Average units sold per day", row["Average units sold per day"]])
    table.add_row(["Average Daily Profit", row["Average Daily Profit"]])
    table.add_row(["Average Daily Profit Without COGS", row["Average Daily Profit Without COGS"]])
 #   table.add_row(["NetProfit", row["NetProfit"]])
    return table.get_string()

### Concatenating Each Row (Price Point) with the Previous Row (Price Point) for Better Comparison

In [10]:
def Remarks(df_S):
    df_t = pd.DataFrame()
    df_final = pd.DataFrame()
    for x in df_S['ASIN'].unique():
        df_t = df_S[df_S['ASIN'] == x]
        df_t.reset_index(inplace = True, drop = True)
        for i in range(0, df_t.shape[0]):
            if i == 0:
                df_t.loc[i,'Remarks'] = None
            else:
                df_t.loc[i,'Remarks'] = generate_pretty_table(df_t.loc[old_i,:]) + generate_pretty_table(df_t.loc[i,:])
            old_i = i
        df_final = pd.concat([df_final, df_t])
        df_final.reset_index(inplace = True)
        df_final.drop(columns = 'index', inplace = True)
    return df_final

# Sales Adjuments report Building

### Inputs

In [11]:
inp = input("Enter your first date in this form:  '2025-02-01'")
period_baseline = int(input("Enter the minimum threshold"))
## Keep only the price points above the threshold
df = df[df['No. of days'] > period_baseline]

Enter your first date in this form:  '2025-02-01'2025-03-01
Enter the minimum threshold0


### Classifying the periods  

In [12]:
pd.options.mode.chained_assignment = None
df = Remarks(df)

In [13]:
numeric_columns = df.select_dtypes(include='float').columns
df[numeric_columns] = df[numeric_columns].round(2)

In [14]:
df_s = pd.DataFrame()
df_t = pd.DataFrame()
for i in df['Marketplace'].unique():
    df_t = df[df['Marketplace'] == i]
    df_t = SalesProfit(df_t)
    df_s = pd.concat([df_t, df_s])
    df_s.reset_index(inplace = True)
    df_s.drop(columns = 'index', inplace = True)

### Data Cleaning part 1

In [15]:
df['Start_Date'] = pd.to_datetime(df['Start_Date'], format = "%d/%m/%Y", errors  = 'ignore')
df['End_Date'] = pd.to_datetime(df['End_Date'], format = "%d/%m/%Y")
df.reset_index(inplace = True)
df.drop(columns = 'index',inplace = True)

<ipython-input-15-b69f2e90e6b1>:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df['Start_Date'] = pd.to_datetime(df['Start_Date'], format = "%d/%m/%Y", errors  = 'ignore')


### Extracting Report template

In [16]:
root_path =  r"/content/drive/My Drive/Sales Data"
os.chdir(os.path.join(root_path))
df_pa = pd.read_excel('Price Adjustments Temp.xlsx',header=[0, 1])

### Data Cleaning part 2

In [17]:
df_s['Month Number'] =df_s['Start_Date'].dt.month
df_s['Remark Number'] = 'Remarks.'+ (df_s['Month Number'] - 1).astype(str)
df_s['week_number'] = df_s['Start_Date'].apply(week_of_month)
df_s['Month Name'] =df_s['Start_Date'].dt.month_name()
df_s['Month Name'] = df_s['Month Name'].str.upper()
for i in range(0, df_s.shape[0]):
    df_s.loc[i,'Sales_num'] = 'Sales Effect.' + str(int(df_s.loc[i, 'week_number'] -1))

df_s['Sales_num'].replace({'Sales Effect.0': 'Sales Effect',
                           'Sales Effect.4': 'Sales Effect.3',
                          'Sales Effect.5': 'Sales Effect.3'},inplace = True)
df_s['week_number'].replace({1: '1st Week', 2:'2nd Week', 3: '3rd Week', 4: '4th-5th Week', 5:'4th-5th Week', 6: '4th-5th Week'}, inplace = True)


df_s.replace( {'nan': None, 'More Profit': "MP", 'Less Profit': 'LP'}, inplace = True)
df_s.replace( {'nan': None, 'More Sales': "MS", 'Less Sales': 'LS'}, inplace = True)
df_s['Remark Number'].replace({'Remarks.0': 'Remarks'}, inplace = True)

df_s['Final_Result'] = df_s['Result in Sales'] + df_s['Result in Profit']
df_s['week_number 2'] =  df_s['week_number'].replace({'4th-5th Week': '4th & 5th Week'})


<ipython-input-17-ada320214d98>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_s['Sales_num'].replace({'Sales Effect.0': 'Sales Effect',
<ipython-input-17-ada320214d98>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tru

### Finalize

In [18]:
#df_s = df_s[df_s['Start_Date'] > inp]
df_s = df_s[df_s['Start_Date'] > inp]

In [19]:
df_main = df_s.copy()
df_s.reset_index(inplace = True)
df_s.drop(columns = 'index', inplace = True)
df_s[['Gross Revenue', 'NetProfit', 'RefundCost', 'FBA storage fee', 'Average units sold per day', 'Expenses',
         'Average Sales per day', 'Average Daily Profit']] = df_s[['Gross Revenue', 'NetProfit', 'RefundCost', 'FBA storage fee', 'Average units sold per day', 'Expenses',
         'Average Sales per day', 'Average Daily Profit']].round(2)

for a in df_main['Marketplace'].unique():
    root_path = r"/content/drive/My Drive/Sales Data"
    os.chdir(root_path)
    df_pa = pd.read_excel('Price Adjustments Temp.xlsx',header=[0, 1])
    df_s = df_main[df_main['Marketplace'] == a]
    df_s.reset_index(inplace = True)
    df_s.drop(columns = 'index', inplace = True)
    count = 0
    for i in range(0, df_s.shape[0]):
        if df_s.loc[i, 'Result in Sales'] == None:
            df_pa.loc[count,df_pa.columns[1]] = df_s.loc[i, 'ASIN']
            df_pa.loc[count,df_pa.columns[0]] = df_s.loc[i, 'PRODUCTS']
            try:
                price = orgin_price[orgin_price['ASIN'] == df_s.loc[i, 'ASIN']]
                price = orgin_price[orgin_price['Marketplace'] == df_s.loc[i, 'Marketplace']]
                df_pa.loc[count,df_pa.columns[2]] = orgin_price['Orginal Price']
            except:
                pass
            count =count +1


    index = df_s[df_s['Result in Sales'].isna()].index
    df_s.drop(index, inplace = True)
    df_s.reset_index(inplace = True)
    df_s.drop(columns = 'index', inplace = True)

    df_t = pd.DataFrame()
    df_pa.set_index(df_pa.columns[1], inplace = True)
    for i in range(0, df_s.shape[0]):
        df_pa.loc[df_s.loc[i, 'ASIN'],(df_s.loc[i, 'Month Name'], df_s.loc[i, 'week_number'])] =df_s.loc[i,"Price"]
        df_pa.loc[df_s.loc[i, 'ASIN'],(df_s.loc[i, 'Month Name'], df_s.loc[i, 'Sales_num'])] = df_s.loc[i,'Final_Result']
        df_pa.loc[df_s.loc[i, 'ASIN'],(df_s.loc[i, 'week_number 2'], df_s.loc[i, 'Remark Number'])] = df_s.loc[i,'Remarks']
    df_pa.reset_index(inplace = True)

    root_path = r"/content/drive/My Drive/Sales Data/Price Adjustments"
    os.chdir(root_path)
    df_pa.to_excel(a + ' '+'Price Adjustments Monitoring.xlsx')

<ipython-input-19-3b65d8833f43>:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_pa.reset_index(inplace = True)
<ipython-input-19-3b65d8833f43>:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_pa.reset_index(inplace = True)
<ipython-input-19-3b65d8833f43>:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = fra